In [9]:
def loadGraph(filename):
    with open(str(filename), "r") as graph_instance:
        chromatic_num = int(graph_instance.readline())
        V, E = list(map(int, graph_instance.readline().split(" ")))
        graph = [[] for _ in range(V)]
        for line in graph_instance:
            if line.startswith('e'):
                line = line.split(" ")
                del line[0]
                edge = list(map(int, line))
                if (edge[1]-1) not in graph[edge[0]-1]:
                    graph[edge[0] - 1].append(edge[1] - 1)
                    graph[edge[1] - 1].append(edge[0] - 1)
    return chromatic_num, graph

In [10]:
def printSolution(color):
    print("Solution: ", end = " ")
    for i in range(len(color)):
        print(str(color[i]), end=" ")
        
    print()

In [11]:
import random
import numpy as np

In [12]:
class Particle:
    
    globalBestPosition = None
    globalBestValue = None
    
    def __init__(self, graph, c_i, c_l, c_g):
        self.graph = graph
        
        self.position = self.initialize()
        self.velocity = 0.0
        
        self.currValue = self.calcSolutionValue()
        self.bestValue = self.currValue
   
        if Particle.globalBestValue is None or self.currValue < Particle.globalBestValue:
            Particle.globalBestPosition = self.position.copy()
            Particle.globalBestValue = self.currValue
        
        self.c_i = c_i
        self.c_l = c_l
        self.c_g = c_g
        
        self.bounds = (1, len(self.graph)+1)
        
        lower_bounds = np.array([self.bounds[0] for _ in range(len(self.position))])
        upper_bounds = np.array([self.bounds[1] for _ in range(len(self.position))])
   
    def initialize(self): 
        solution = []
        V = len(self.graph)
    
        for i in range(V):
            solution.append(random.randrange(1, V+1))
    
        if not self.isFeasible(solution):
            self.repair(solution)
        return np.array(solution)
        
    def isFeasible(self, solution):
        for i in range(len(self.graph)):
            for j in self.graph[i]:
                if(solution[i] == solution[j]):
                    return False
        return True
    
    def repair(self, solution):
        V = len(self.graph)

        for i in range(V):
            neighbors = [index for index in self.graph[i]]
            used_colors = [solution[neighbor] for neighbor in neighbors]
            if(solution[i] in used_colors):  
                available_colors = [c for c in range(1, V+1) if c not in used_colors]
                solution[i] = random.choice(available_colors)

    def calcSolutionValue(self):
        return len(set(self.position))
    
    def updatePosition(self):
        lower_bounds = np.array([self.bounds[0] for _ in range(len(self.position))])
        upper_bounds = np.array([self.bounds[1] for _ in range(len(self.position))])
        
        self.position = np.clip(self.position + int(self.velocity), lower_bounds, upper_bounds)    
            
        if not self.isFeasible(self.position):
            self.repair(self.position)
        
        self.currValue = self.calcSolutionValue()
        if self.currValue < self.bestValue:
            self.bestValue = self.currValue
            if self.currValue < Particle.globalBestValue:
                Particle.globalBestValue = self.currValue
                Particle.globalBestPosition = self.position.copy()
                   
    def updateVelocity(self):
        r_l = random.random()
        r_g = random.random()
        self.velocity = (self.c_i * self.velocity
                      + r_l * self.c_l * (self.bestValue - self.currValue) # cognitive 
                      + r_g * self.c_g * (Particle.globalBestValue - self.currValue)) # social 
        
        if self.velocity > len(self.graph):
            self.velocity = len(self.graph)
        elif self.velocity < -len(self.graph):
            self.velocity = -len(self.graph)

In [15]:
def PSO(graph, swarm_size, max_iters, start):
    MAX_TIME_EXEC = 1800
    Particle.globalBestValue = float("inf")
    swarm = [Particle(graph, 0.75, 1.8, 2) for _ in range(swarm_size)]
    end_interupt = -1
    for i in range(max_iters):
        for j, particle in enumerate(swarm):
            particle.updateVelocity()
            particle.updatePosition()
            
            if not particle.isFeasible(particle.position):
                print("greska")
                
        if Particle.globalBestValue == chromatic_num:
            break
            
        end_interupt = time.time()
        if (end_interupt-start) > MAX_TIME_EXEC:
            break
            
            
    return Particle.globalBestValue, Particle.globalBestPosition, i+1, end_interupt

In [16]:
graph_list = ['brute.txt', 'myciel3.col', 'myciel4.col', 'myciel5.col', 'myciel6.col', 'david.col', 'huck.col', 'jean.col']
import time
from termcolor import colored

for max_iters in [100, 1000, 10000]:
    print(f'---------------------Max iters: {max_iters}---------------------')
    for graph_file in graph_list:
                print()
                print(colored(graph_file, color="red"))
                graph = []
                chromatic_num, graph = loadGraph(graph_file)
                
                start = time.time()
                num_of_colors, color, num_of_iters, end_interupt = PSO(graph, 100, max_iters, start)
                end = time.time()
                
                if end_interupt > 0:
                    end = end_interupt
                
                exec_time = end - start
                
                print('time: ', exec_time)
                printSolution(color)
                print(f'Chromatic number: {chromatic_num}, Found chromatic number: {num_of_colors}')
                print(f'Number of iters: {num_of_iters}')

---------------------Max iters: 100---------------------

brute.txt
time:  0.007664680480957031
Solution:  5 3 1 2 1 
Chromatic number: 4, Found chromatic number: 4
Number of iters: 1

myciel3.col
time:  0.5999951362609863
Solution:  5 2 4 2 6 5 1 1 1 1 6 
Chromatic number: 4, Found chromatic number: 5
Number of iters: 100

myciel4.col
time:  1.901482105255127
Solution:  9 8 2 18 9 4 8 4 3 3 5 1 1 1 7 1 3 1 3 10 1 1 2 
Chromatic number: 5, Found chromatic number: 10
Number of iters: 100

myciel5.col
time:  7.1172356605529785
Solution:  5 15 40 28 14 8 13 8 11 9 5 8 38 11 30 30 14 20 13 13 43 36 21 1 1 1 1 1 1 1 1 1 2 15 1 1 1 1 3 1 12 1 1 1 14 1 4 
Chromatic number: 6, Found chromatic number: 20
Number of iters: 100

myciel6.col
time:  42.69297194480896
Solution:  20 51 9 28 2 76 16 38 46 14 4 72 17 36 3 62 40 26 17 72 41 8 61 12 30 22 6 12 88 25 46 27 19 25 14 90 11 18 2 15 41 72 23 45 4 86 28 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 11 1 9 1 1 1 1 1 1 1 1 1 1 1